In [150]:
import json
import itertools
import pandas as pd
import ast
import string
import operator
from collections import defaultdict
import re
import os
import nltk
import math
import json
from multiprocessing import Pool
from os import listdir
from os.path import isfile, join
from os import walk
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
!cd /Users/sdatta/wikipedia_w251/wikipedia/json_out



In [ ]:
!sh parse.sh

In [120]:
df_wiki_meta= pd.read_csv('/Users/sdatta/wikipedia_w251/parse.sh.nohup.out',names=['filename','id','url','title'], delimiter='|')
#df_wiki_meta.columns=['filename','id','url','title']
##TODO convert 'id' to integer

In [121]:
root_dir='/Users/sdatta/wikipedia_w251'
output_psv='df_wiki_meta.psv'
df_wiki_meta.to_csv(root_dir+'/'+output_psv,sep='|', index=False)

In [122]:
df_wiki_meta['categories']=''

In [10]:
import requests

url="https://en.wikipedia.org/w/api.php"
params = dict(
    format='json',
    action='query',
    prop='categories',
    titles=''
)


def get_categories(in_title):
    ''' Makes a call to https://en.wikipedia.org/w/api.php with the article title to retrieve the corresponding categories.
    Populates list categ_list with the categories.
    Returns categ_list.
    '''
    params['titles']=in_title
    #url="https://en.wikipedia.org/w/api.php?format=json&action=query&prop=categories&titles=Google"
    resp = requests.get(url=url, params=params)
    data = resp.json() 
    categ_list=[]
    for ele1 in data['query']['pages']:
        try:
            for ele2 in data['query']['pages'][ele1]['categories'][1:]:
                categ_list.append(ele2['title'].split(':')[1])
        except:
            print("Exception in in_title",in_title," ele1",ele1," data['query']['pages']",data['query']['pages'])
            
    #categ_list='~'.join(categ_list)
    return categ_list

def apply_categories(in_df):
    in_df['categories']=df_wiki_meta['title'].apply(get_categories)
    return in_df
    

In [103]:
get_categories('Anarchism')

['All articles with unsourced statements',
 'Anarchism',
 'Anti-capitalism',
 'Anti-fascism',
 'Articles containing Spanish-language text',
 'Articles lacking reliable references from March 2019',
 'Articles prone to spam from November 2014',
 'Articles with short description',
 'Articles with unsourced statements from March 2019']

In [105]:
root='/Users/sdatta/wikipedia_w251'
sub_dir='categ_out/AA'
files_to_process=['AA/wiki_00']
# sub_dir="categ_out/AG"
# files_to_process=['AG/wiki_00','AG/wiki_01','AG/wiki_02','AG/wiki_03','AG/wiki_04','AG/wiki_05','AG/wiki_06','AG/wiki_07','AG/wiki_08','AG/wiki_09','AG/wiki_10','AG/wiki_11','AG/wiki_12','AG/wiki_13','AG/wiki_14','AG/wiki_15','AG/wiki_16','AG/wiki_17','AG/wiki_18','AG/wiki_19','AG/wiki_20','AG/wiki_21','AG/wiki_22','AG/wiki_23','AG/wiki_24','AG/wiki_25','AG/wiki_26','AG/wiki_27','AG/wiki_28','AG/wiki_29','AG/wiki_30','AG/wiki_31','AG/wiki_32','AG/wiki_33','AG/wiki_34','AG/wiki_35','AG/wiki_36','AG/wiki_37','AG/wiki_38','AG/wiki_39','AG/wiki_40','AG/wiki_41','AG/wiki_42','AG/wiki_43','AG/wiki_44','AG/wiki_45','AG/wiki_46','AG/wiki_47','AG/wiki_48','AG/wiki_49','AG/wiki_50','AG/wiki_51','AG/wiki_52','AG/wiki_53','AG/wiki_54','AG/wiki_55''AG/wiki_56','AG/wiki_57','AG/wiki_58','AG/wiki_59','AG/wiki_60','AG/wiki_61','AG/wiki_62','AG/wiki_63','AG/wiki_64','AG/wiki_65','AG/wiki_66','AG/wiki_67','AG/wiki_68','AG/wiki_69','AG/wiki_70','AG/wiki_71','AG/wiki_72','AG/wiki_73','AG/wiki_74','AG/wiki_75','AG/wiki_76','AG/wiki_77','AG/wiki_78','AG/wiki_79','AG/wiki_80','AG/wiki_81','AG/wiki_82','AG/wiki_83','AG/wiki_84','AG/wiki_85','AG/wiki_86','AG/wiki_87','AG/wiki_88','AG/wiki_89','AG/wiki_90','AG/wiki_91','AG/wiki_92','AG/wiki_93','AG/wiki_94','AG/wiki_95','AG/wiki_96','AG/wiki_97','AG/wiki_98','AG/wiki_99']

In [ ]:
#Working good
#
#Calls function get_categories for each value of title in df_wiki_meta_temp/df_wiki_meta
#

for filename in files_to_process:
    pkl_filename=root+'/'+sub_dir+"/"+filename.replace('/','_')+'.psv'
    df_wiki_meta_temp=pd.DataFrame()
    df_wiki_meta_temp=df_wiki_meta.loc[df_wiki_meta['filename'].str.contains(filename, na=False)].copy()
    df_wiki_meta_temp['categories'] = df_wiki_meta_temp['title'].apply(get_categories)
    df_wiki_meta_temp.to_csv(pkl_filename,sep='|',index=False)



In [136]:
files_to_process

['AA/wiki_00']

In [121]:
# mypath='/Users/sdatta/wikipedia_w251/wikipedia/json_out'
# filelist=[]
# for path, subdirs, files in os.walk(mypath):
#     for name in files:
#         filelist.append(os.path.join(path, name))


In [20]:
df_wiki_meta_temp.shape

(8, 5)

# Adhoc loading of AA/wiki_00 file

## Question_id to title mapping

In [125]:
#
#wiki_squad_00.json shared by Julia
#Creates a map of title to question id from wiki_squad_00.json
#
rootdir="/Users/sdatta/Documents/1. Personal/MIDS/W210 Capstone/w210-capstone-qg"
input_json='./wiki_squad_00.json'
list_title_qid=[]


with open(rootdir+"/"+input_json) as json_file:  
    input_dict=json.load(json_file)
    
for ele1 in input_dict['data']:
    title=ele1['title']
    for ele2 in ele1['paragraphs']:
        for ele3 in ele2['qas']:
            qid=tuple(ele3['id'][1:-1].replace(" ", "").split(','))
            list_title_qid.append((title, qid ))
    


In [126]:
df_title_qid=pd.DataFrame(list_title_qid,columns=['title','qid'], index=None)  


In [127]:
df_title_qid.shape

(367, 2)

In [128]:
df_title_qid_AA_00=df_title_qid.copy()

In [129]:
df_title_qid_AA_00.shape

(367, 2)

In [138]:
rootdir="/Users/sdatta/wikipedia_w251/categ_out"
input_psv='AA/AA_wiki_00.psv'
df_categories_AA=pd.read_csv(rootdir+"/"+input_psv, sep='|'
                                )

In [131]:
df_categories_AA.columns

Index(['filename', 'id', 'url', 'title', 'categories'], dtype='object')

In [139]:
#
#Joins df_title_qid_AA_00 with df_categories_AA 
#
df_title_qid_AA_00_merged=pd.merge(df_title_qid_AA_00,df_categories_AA, how='left', left_on='title', right_on='title'  )

In [140]:
df_title_qid_AA_00_merged['id']=df_title_qid_AA_00_merged['id'].apply(pd.to_numeric)

In [141]:
print(df_title_qid_AA_00.columns)
print(df_wiki_meta.columns)
print(df_title_qid_AA_00_merged.columns)
print(df_title_qid_AA_00_merged.shape)

Index(['title', 'qid'], dtype='object')
Index(['filename', 'id', 'url', 'title', 'categories'], dtype='object')
Index(['title', 'qid', 'filename', 'id', 'url', 'categories'], dtype='object')
(367, 6)


In [142]:
rootdir="/Users/sdatta/wikipedia_w251/categ_out"
output_psv='./df_title_qid_AA_00_merged.psv'
df_title_qid_AA_00_merged.to_csv(rootdir+"/"+output_psv, sep='|', index=False
                                )